In [1]:
import psycopg2
import pandas as pd 
# from sqlalchemy.types import Integer, Text, String, DateTime
import sqlalchemy as salc
import matplotlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json


with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = salc.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [2]:
repo_list = pd.DataFrame()
repo_list_query = f"""
SELECT
	* 
FROM
	(
		( SELECT * FROM repo ) repo
		LEFT JOIN (
		SELECT
			RH.*,
			committer.*,
			CAST ( ( CEIL ( RH.RH_commits + 1 ) :: NUMERIC / ( CEIL ( committer.all_commits + 1 ) :: NUMERIC ) ) AS NUMERIC ) AS commit_pct,
			CAST ( ( CEIL ( RH.RH_added + 1 ) :: NUMERIC / ( CEIL ( committer.all_added + 1 ) :: NUMERIC ) ) AS NUMERIC ) AS added_pct,
			CAST ( ( CEIL ( RH.RH_deleted + 1 ) :: NUMERIC / ( CEIL ( committer.all_deleted + 1 ) :: NUMERIC ) ) AS NUMERIC ) AS deleted_pct,
			CAST ( ( CEIL ( RH.RH_whitespace + 1 ) :: NUMERIC / ( CEIL ( committer.all_whitespace + 1 ) ) :: NUMERIC ) AS NUMERIC ) AS whitespace_pct 
		FROM
			(
			SELECT
				b.repo_id AS all_repos_id,
				COUNT ( C.cmt_id ) AS all_commits,
				SUM ( C.cmt_added ) AS all_added,
				SUM ( C.cmt_removed ) AS all_deleted,
				SUM ( C.cmt_whitespace ) AS all_whitespace 
			FROM
				repo_groups A,
				repo b,
				commits C 
			WHERE
				A.repo_group_id = b.repo_group_id 
				AND b.repo_id = C.repo_id 
			GROUP BY
				A.repo_group_id,
				b.repo_id 
			ORDER BY
				A.repo_group_id,
				b.repo_id 
			) committer
			LEFT JOIN (
			SELECT
				b.repo_id AS RH_repos_id,
				COUNT ( C.cmt_id ) AS RH_commits,
				SUM ( C.cmt_added ) AS RH_added,
				SUM ( C.cmt_removed ) AS RH_deleted,
				SUM ( C.cmt_whitespace ) AS RH_whitespace 
			FROM
				repo_groups A,
				repo b,
				commits C 
			WHERE
				A.repo_group_id = b.repo_group_id 
				AND b.repo_id = C.repo_id 
				AND (
					C.cmt_author_email LIKE'%pivotal.com' 
					OR C.cmt_author_email LIKE '%vmware.com' 
					OR C.cmt_author_email LIKE '%rabbitmq.com' 
					OR C.cmt_author_email LIKE '%pivotallabs.com' 
					OR C.cmt_author_email LIKE '%cloudcredo.com' 
					OR C.cmt_author_email LIKE '%gopivotal.com' 
					OR C.cmt_author_email LIKE '%heptio.com' 
				) 
			GROUP BY
				A.repo_group_id,
				b.repo_id 
			ORDER BY
				A.repo_group_id,
				b.repo_id 
			) RH ON committer.all_repos_id = RH.RH_repos_id 
		ORDER BY
			committer.all_repos_id,
			RH.RH_repos_id 
		) allcommits ON repo.repo_id = allcommits.all_repos_id 
	) pdd;
    """

#repo_list = pd.read_sql_query(repo_list_query, con=engine)
salcQ = salc.text(repo_list_query)
repo_list = pd.read_sql_query(salcQ, con=engine)

print(repo_list)

      repo_id  repo_group_id  \
0       25430          25155   
1       25431          25155   
2       25432          25155   
3       25433          25155   
4       25434          25155   
...       ...            ...   
2706    27837          25180   
2707    26072          25164   
2708    27603          25179   
2709    26834          25171   
2710    25948          25160   

                                               repo_git  \
0                   https://github.com/rabbitmq/rmq-0mq   
1                https://github.com/rabbitmq/rabbitmq-c   
2           https://github.com/rabbitmq/rabbitmq-server   
3          https://github.com/rabbitmq/rabbitmq-codegen   
4              https://github.com/rabbitmq/rabbitmq-bql   
...                                                 ...   
2706  https://github.com/pivotal-cf-experimental/geo...   
2707     https://github.com/cfmobile/cfmobile.github.io   
2708       https://github.com/pivotal-cf/docs-app-tools   
2709  https://github.com/